## Score the Organizations Using the Name, Alternative Name, and Previous Name Fields

Scores the matches between the PatentsView and OpenCorporates data sets using the file generated via the Prepare OpenCorporates Data with No States script. The resulting scores are utilized to determine the confidence levels for each record and bound between 1 and 10. If any duplications exist, they are dropped and the first record is selected as the representative record.

In [1]:
### import the libraries used to process the PatentsView and OC data.
import pandas as pd
import numpy as np
import time
import os
import re
import string
import warnings
warnings.filterwarnings('ignore')

### start timer
t0=time.time()

### set the path for the input file and save to variable
res_folder = "../csvResults/"
input_file = "OcResults1000WithUniqueAssigneeIDsAddCoorPreparedForScoring.csv"
a_full=os.path.join(res_folder,input_file)

OCScoringDF=pd.read_csv(a_full)

### end timer and print total time
t1 = time.time()
total = t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset 
display(OCScoringDF.info(null_counts=True),OCScoringDF.head())

Total time is 0.000000 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   378 non-null    int64  
 1   assignee_id          378 non-null    object 
 2   location_id          378 non-null    object 
 3   organization         378 non-null    object 
 4   city                 378 non-null    object 
 5   state                378 non-null    object 
 6   city_latitude        374 non-null    float64
 7   city_longitude       374 non-null    float64
 8   dateOfFirstPat       378 non-null    object 
 9   serial               36 non-null     object 
 10  tradeCity            36 non-null     object 
 11  tradeState           36 non-null     object 
 12  ctrlEntity           36 non-null     object 
 13  nameScores           378 non-null    float64
 14  matchNames           378 non-null    object 
 15  subJurisCode

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,serial,...,agent_latitude,agent_longitude,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,1999-01-20,NaN,...,37.2738,-79.9602,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.08
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2017-01-12,NaN,...,41.8171,-71.4282,NaN,NaN,NaN,NaN,0.0,49.5,NaN,0.02
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2012-12-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09
3,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,36.1716,-115.1391,1980-10-23,NaN,...,NaN,NaN,Las Vegas,Nv,36.1716,-115.1391,0.0,NaN,0.0,22.50
4,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2014-01-15,NaN,...,NaN,NaN,NaN,Dc,NaN,NaN,NaN,NaN,NaN,1.04


## Scoring Algorithm

In [2]:
totalScore=[]
d=len(OCScoringDF)
x=''

### score the fuzzy match percentages

for s in range(d):
    ### scores all records with a fuzzy match score with a 100%; account for organization
    ### name lengths (i.e., shorter names are scored lower than longer names)
    
    if OCScoringDF['nameScores'][s] == 100:

        if len(OCScoringDF['organization'][s]) < 5:
            x=0
        
        elif 5 <= len(OCScoringDF['organization'][s]) < 10:
            x=3
            
        elif len(OCScoringDF['organization'][s]) >= 10:
            x=5
    
    ### visual inspection of the data indicates a discrete group between 95% and 100%,
    ### resulting in the next set. Name lengths are again accounted for and shorter
    ### names/scores are downweighted even more
    
    elif 95 <= OCScoringDF['nameScores'][s] < 100:
        
        if len(OCScoringDF['organization'][s]) < 5:
            x=-1
        
        elif 5 <= len(OCScoringDF['organization'][s]) < 10:
            x=1
            
        elif 10 <= len(OCScoringDF['organization'][s]) < 15:
            x=2
            
        elif len(OCScoringDF['organization'][s]) >= 15:
            x=3

    ### many of the fuzzy matches in this range are wrong, but there are a couple correct
    ### that should not be discounted. While the weights are not as high as the previous
    ### sections, correct matches will be given better scores than the next section
    
    elif 90 <= OCScoringDF['nameScores'][s] < 95:
        
        if len(OCScoringDF['organization'][s]) < 5:
            x=-2
        
        elif 5 <= len(OCScoringDF['organization'][s]) < 10:
            x=1
            
        elif 10 <= len(OCScoringDF['organization'][s]) < 15:
            x=2
            
        elif len(OCScoringDF['organization'][s]) >= 15:
            x=3
            
    elif OCScoringDF['nameScores'][s] < 90:
        
        if len(OCScoringDF['organization'][s]) < 5:
            x=-3
        
        elif 5 <= len(OCScoringDF['organization'][s]) < 10:
            x=0
            
        elif 10 <= len(OCScoringDF['organization'][s]) < 15:
            x=1
            
        elif len(OCScoringDF['organization'][s]) >= 15:
            x=2

            
    ### the code below searches for a match between the PatentsView and trademark states against the
    ### OpenCorporates states. The initial search attempts to match the PatentsView state with the
    ### subJurisCode; if there is no match, then the subCntlEntity feature was used to match against
    ### the state. Since data set represents data that is not restricted to state, there are many
    ### records that do not have a matching state. In these cases, the next step is to use the
    ### trademark states and match to the same OpenCorporates states. If none of these cases are
    ### satisfied, the states under the address_, agent_, and data_ fields were searched. Records
    ### with no matches will recieve a low score.
    
    if OCScoringDF['state'][s] == OCScoringDF['subJurisCode'][s]:
        x=x+5
        
        if OCScoringDF['city'][s] == OCScoringDF['address_city'][s]:
            x=x+5

        elif OCScoringDF['city'][s] == OCScoringDF['agent_city'][s]:
            x=x+5

        elif OCScoringDF['city'][s] == OCScoringDF['data_city'][s]:
            x=x+5
        

    elif OCScoringDF['state'][s] == OCScoringDF['subCntlEntity'][s]:
        x=x+5
        
        if OCScoringDF['city'][s] == OCScoringDF['address_city'][s]:
            x=x+5

        elif OCScoringDF['city'][s] == OCScoringDF['agent_city'][s]:
            x=x+5

        elif OCScoringDF['city'][s] == OCScoringDF['data_city'][s]:
            x=x+5
    

    elif OCScoringDF['tradeState'][s] == OCScoringDF['subJurisCode'][s]:
        x=x+5
        
        if OCScoringDF['tradeCity'][s] == OCScoringDF['address_city'][s]:
            x=x+5

        elif OCScoringDF['tradeCity'][s] == OCScoringDF['agent_city'][s]:
            x=x+5

        elif OCScoringDF['tradeCity'][s] == OCScoringDF['data_city'][s]:
            x=x+5
    

    elif OCScoringDF['tradeState'][s] == OCScoringDF['subCntlEntity'][s]:
        x=x+5
        
        if OCScoringDF['tradeCity'][s] == OCScoringDF['address_city'][s]:
            x=x+5

        elif OCScoringDF['tradeCity'][s] == OCScoringDF['agent_city'][s]:
            x=x+5

        elif OCScoringDF['tradeCity'][s] == OCScoringDF['data_city'][s]:
            x=x+5
        

    elif OCScoringDF['ctrlEntity'][s] == OCScoringDF['subJurisCode'][s]:
        x=x+5
    

    elif OCScoringDF['ctrlEntity'][s] == OCScoringDF['subCntlEntity'][s]:
        x=x+5


    if x < 8:
        
        if OCScoringDF['state'][s] == OCScoringDF['address_state'][s]:
            x=x+5
        
            if OCScoringDF['city'][s] == OCScoringDF['address_city'][s]:
                x=x+5

        elif OCScoringDF['state'][s] == OCScoringDF['agent_state'][s]:
            x=x+5
        
            if OCScoringDF['city'][s] == OCScoringDF['agent_city'][s]:
                x=x+5
        
        elif OCScoringDF['state'][s] == OCScoringDF['data_state'][s]:
            x=x+5
            
            if OCScoringDF['city'][s] == OCScoringDF['data_city'][s]:
                x=x+5

    ### the code below penalizes records with a first patent applied for date that is older
    ### than the incorporation date for that organization
    
    if OCScoringDF['dateOfFirstPat'][s] < OCScoringDF['minIncDateForOrg'][s]:
        
        if OCScoringDF['dateDiff'][s] > 1:
            x=x-5
        
    ### below scores the distances between the PatentsView location and the closest location reported
    ### by OpenCorporates. The input file uses miles as the unit and the code below evaluates distances
    ### in intervals.    

    if (OCScoringDF.iloc[s,31] == 0 or OCScoringDF.iloc[s,32] == 0 or OCScoringDF.iloc[s,33] == 0):
        x=x+5
        
    elif (0 < OCScoringDF.iloc[s,31] < 10 or 0 < OCScoringDF.iloc[s,32] < 10 or 0 < OCScoringDF.iloc[s,33] < 10):
        x=x+4
    
    elif (10 <= OCScoringDF.iloc[s,31] < 50 or 10 <= OCScoringDF.iloc[s,32] < 50 or 10 <= OCScoringDF.iloc[s,33] < 50):
        x=x+3
        
    elif (50 <= OCScoringDF.iloc[s,31] < 100 or 50 <= OCScoringDF.iloc[s,32] < 100 or 50 <= OCScoringDF.iloc[s,33] < 100):
        x=x+2
        
    elif (100 <= OCScoringDF.iloc[s,31] < 200 or 100 <= OCScoringDF.iloc[s,32] < 200 or 100 <= OCScoringDF.iloc[s,33] < 200):
        x=x+1
    
    elif (OCScoringDF.iloc[s,31] >= 200 or OCScoringDF.iloc[s,32] >= 200 or OCScoringDF.iloc[s,33] >= 200):
        x=x-2
    
    ### lastly, the time differences between the incorporation date and first patent application was
    ### evaluate and scored using the intervals below. Time from the input file is in years.
    
#     try:
    if OCScoringDF['dateDiff'][s] <= 15:
        x=x+5

    elif 15 < OCScoringDF['dateDiff'][s] <= 25:
        x=x+4

    elif 25 < OCScoringDF['dateDiff'][s] <= 30:
        x=x+3

    elif 30 < OCScoringDF['dateDiff'][s] <= 35:
        x=x+2

    elif OCScoringDF['dateDiff'][s] > 35:
        x=x+1

#     except:
#         pass
    
    totalScore.append(x)

In [3]:
OCScoringDF['totalScore'] = totalScore

display(OCScoringDF.info(),OCScoringDF.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   378 non-null    int64  
 1   assignee_id          378 non-null    object 
 2   location_id          378 non-null    object 
 3   organization         378 non-null    object 
 4   city                 378 non-null    object 
 5   state                378 non-null    object 
 6   city_latitude        374 non-null    float64
 7   city_longitude       374 non-null    float64
 8   dateOfFirstPat       378 non-null    object 
 9   serial               36 non-null     object 
 10  tradeCity            36 non-null     object 
 11  tradeState           36 non-null     object 
 12  ctrlEntity           36 non-null     object 
 13  nameScores           378 non-null    float64
 14  matchNames           378 non-null    object 
 15  subJurisCode         378 non-null    obj

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,serial,...,agent_longitude,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,1999-01-20,NaN,...,-79.9602,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.08,25
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2017-01-12,NaN,...,-71.4282,NaN,NaN,NaN,NaN,0.0,49.5,NaN,0.02,25
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2012-12-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,8
3,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,36.1716,-115.1391,1980-10-23,NaN,...,NaN,Las Vegas,Nv,36.1716,-115.1391,0.0,NaN,0.0,22.50,24
4,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2014-01-15,NaN,...,NaN,NaN,Dc,NaN,NaN,NaN,NaN,NaN,1.04,13


In [4]:
OCScoringDF['assignee_id'].nunique()

378

In [5]:
OCScoringDF['confidenceScore']=((10-1)*((OCScoringDF['totalScore']-min(OCScoringDF['totalScore']))/
                                         (max(OCScoringDF['totalScore'])-min(OCScoringDF['totalScore']))))+1

OCScoringDF['confidenceScore']=[round(num1, 2) for num1 in OCScoringDF['confidenceScore']]

display(OCScoringDF.info(),OCScoringDF.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   378 non-null    int64  
 1   assignee_id          378 non-null    object 
 2   location_id          378 non-null    object 
 3   organization         378 non-null    object 
 4   city                 378 non-null    object 
 5   state                378 non-null    object 
 6   city_latitude        374 non-null    float64
 7   city_longitude       374 non-null    float64
 8   dateOfFirstPat       378 non-null    object 
 9   serial               36 non-null     object 
 10  tradeCity            36 non-null     object 
 11  tradeState           36 non-null     object 
 12  ctrlEntity           36 non-null     object 
 13  nameScores           378 non-null    float64
 14  matchNames           378 non-null    object 
 15  subJurisCode         378 non-null    obj

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,serial,...,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore,confidenceScore
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,1999-01-20,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.08,25,10.0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2017-01-12,NaN,...,NaN,NaN,NaN,NaN,0.0,49.5,NaN,0.02,25,10.0
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2012-12-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,8,4.9
3,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,36.1716,-115.1391,1980-10-23,NaN,...,Las Vegas,Nv,36.1716,-115.1391,0.0,NaN,0.0,22.50,24,9.7
4,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2014-01-15,NaN,...,NaN,Dc,NaN,NaN,NaN,NaN,NaN,1.04,13,6.4


In [6]:
# OCScoringDF.to_csv("../csvResults/aScoredData.csv",index=False)

In [7]:
col         = 'confidenceScore'
conditions  = [ (OCScoringDF[col] >= 1) & (OCScoringDF[col] < 2),
                (OCScoringDF[col] >= 2) & (OCScoringDF[col] < 3),
                (OCScoringDF[col] >= 3) & (OCScoringDF[col] < 4),
                (OCScoringDF[col] >= 4) & (OCScoringDF[col] < 5),
                (OCScoringDF[col] >= 5) & (OCScoringDF[col] < 6),
                (OCScoringDF[col] >= 6) & (OCScoringDF[col] < 7),
                (OCScoringDF[col] >= 7) & (OCScoringDF[col] < 8),
                (OCScoringDF[col] >= 8) & (OCScoringDF[col] < 9),          
                (OCScoringDF[col] >= 9) & (OCScoringDF[col] < 10),
                (OCScoringDF[col] == 10) ]
choices     = [ 1,1,1,1,1,9,8,8,9,10 ]
    
OCScoringDF["score"] = np.select(conditions, choices, default=np.nan)
OCScoringDF.drop(columns=['confidenceScore'],inplace=True)
OCScoringDF3=OCScoringDF.sort_values(by=['ID']).reset_index(drop=True)

display(OCScoringDF3.info(),OCScoringDF3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   378 non-null    int64  
 1   assignee_id          378 non-null    object 
 2   location_id          378 non-null    object 
 3   organization         378 non-null    object 
 4   city                 378 non-null    object 
 5   state                378 non-null    object 
 6   city_latitude        374 non-null    float64
 7   city_longitude       374 non-null    float64
 8   dateOfFirstPat       378 non-null    object 
 9   serial               36 non-null     object 
 10  tradeCity            36 non-null     object 
 11  tradeState           36 non-null     object 
 12  ctrlEntity           36 non-null     object 
 13  nameScores           378 non-null    float64
 14  matchNames           378 non-null    object 
 15  subJurisCode         378 non-null    obj

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,serial,...,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore,score
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,1999-01-20,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.08,25,10.0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2017-01-12,NaN,...,NaN,NaN,NaN,NaN,0.0,49.5,NaN,0.02,25,10.0
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2012-12-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,8,1.0
3,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,36.1716,-115.1391,1980-10-23,NaN,...,Las Vegas,Nv,36.1716,-115.1391,0.0,NaN,0.0,22.50,24,9.0
4,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2014-01-15,NaN,...,NaN,Dc,NaN,NaN,NaN,NaN,NaN,1.04,13,9.0


In [8]:
finalScoresWoStates=OCScoringDF3.iloc[:,[0,1,2,3,4,5,8,10,11,12,13,14,15,16,17,18,19,20,
                                         23,24,27,28,35,36]].sort_values(by=['ID'])

display(finalScoresWoStates.info(),finalScoresWoStates.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 0 to 377
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   378 non-null    int64  
 1   assignee_id          378 non-null    object 
 2   location_id          378 non-null    object 
 3   organization         378 non-null    object 
 4   city                 378 non-null    object 
 5   state                378 non-null    object 
 6   dateOfFirstPat       378 non-null    object 
 7   tradeCity            36 non-null     object 
 8   tradeState           36 non-null     object 
 9   ctrlEntity           36 non-null     object 
 10  nameScores           378 non-null    float64
 11  matchNames           378 non-null    object 
 12  subJurisCode         378 non-null    object 
 13  subCntlEntity        143 non-null    object 
 14  minIncDateForOrg     378 non-null    object 
 15  minIncDateForOrgLoc  378 non-null    obj

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,tradeCity,tradeState,ctrlEntity,...,minIncDateForOrg,minIncDateForOrgLoc,address_city,address_state,agent_city,agent_state,data_city,data_state,totalScore,score
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1999-01-20,NaN,NaN,NaN,...,1997-12-22,12/22/1997,Roanoke,Va,Roanoke,Va,NaN,NaN,25,10.0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,2017-01-12,NaN,NaN,NaN,...,2017-01-03,01/08/2019,Vineyard Haven,Ma,Providence,Ri,NaN,NaN,25,10.0
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,2012-12-28,NaN,NaN,NaN,...,2013-01-31,01/31/2013,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0
3,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,1980-10-23,NaN,NaN,NaN,...,1958-05-01,05/01/1958,Las Vegas,Nv,NaN,NaN,Las Vegas,Nv,24,9.0
4,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,2014-01-15,NaN,NaN,NaN,...,2013-01-02,09/02/2016,NaN,NaN,NaN,NaN,NaN,Dc,13,9.0


In [9]:
finalScoresWoStates['assignee_id'].nunique()

378

In [10]:
finalScoresWoStates['score'].value_counts().sort_index(ascending=False)

10.0     51
9.0      44
8.0      79
1.0     204
Name: score, dtype: int64

In [11]:
### save the new dataset as a checkpoint
res_folder = "../csvResults/"
outpt_file = "OcResults1000ScoredData.csv"
a_full = os.path.join(res_folder,outpt_file)

# finalScoresWoStates.to_csv(a_full,index=False)